# CIFAR-10 Diffusion - RGB Images Here We Come! 🚀

Time to level up from grayscale MNIST to full-color CIFAR-10! This notebook demonstrates our generic diffusion pipeline working on **32×32 RGB images** with minimal changes to the architecture.

## Load the CIFAR-10 Dataset

CIFAR-10 contains 50,000 training images across 10 classes: airplane, automobile, bird, cat, deer, dog, frog, horse, ship, and truck. Each image is **32×32 pixels with 3 color channels (RGB)**.

In [ ]:
import torch
import matplotlib.pyplot as plt
from torchvision import datasets, transforms

from src.diffusion_playground.diffusion.noise_schedule import LinearNoiseSchedule
from src.diffusion_playground.diffusion.training_utils import sample_xt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

## Explore the Dataset

In [ ]:
# Load CIFAR-10 dataset directly as a tensor
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize RGB channels to [-1, 1]
])

cifar_dataset = datasets.CIFAR10(root="data", train=True, transform=transform, download=True)

# Extract all images into a single tensor
cifar_data = torch.stack([cifar_dataset[i][0] for i in range(len(cifar_dataset))])
cifar_labels = torch.tensor([cifar_dataset[i][1] for i in range(len(cifar_dataset))])

print(f"Dataset shape: {cifar_data.shape}")
print(f"Labels shape: {cifar_labels.shape}")
print(f"Input shape per image: (3, 32, 32)")
print(f"\nClasses: {cifar_dataset.classes}")

In [ ]:
# Visualize a few random samples
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
for idx, ax in enumerate(axes.flat):
    sample_idx = torch.randint(0, len(cifar_data), (1,)).item()

    # Convert from [-1, 1] to [0, 1] for visualization
    img = (cifar_data[sample_idx].permute(1, 2, 0) + 1) / 2
    img = torch.clamp(img, 0, 1)  # Ensure values are in valid range

    ax.imshow(img.cpu())
    ax.set_title(f"{cifar_dataset.classes[cifar_labels[sample_idx]]}")
    ax.axis("off")

plt.suptitle("Sample CIFAR-10 Images", fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## Forward Diffusion on CIFAR-10

Let's visualize how images progressively become noise during the forward diffusion process. We'll see the same image at different time steps from clean (t=0) to pure noise (t=999).

In [ ]:
# Create NoiseSchedule
schedule = LinearNoiseSchedule(time_steps=1_000)

# Use a batch from our data for visualization
x0_batch = cifar_data[:128]  # Take first 128 samples

time_steps = [0, 50, 200, 500, 999]

plt.figure(figsize=(15, 3))
for i, t in enumerate(time_steps):
    # Create a batch of time steps (one per sample in the batch)
    t_tensor = torch.full((x0_batch.shape[0],), t)
    xt, _, _ = sample_xt(x0_batch, schedule, t=t_tensor)

    # Convert from [-1, 1] to [0, 1] for visualization
    img = (xt[0].permute(1, 2, 0) + 1) / 2
    img = torch.clamp(img, 0, 1)

    plt.subplot(1, len(time_steps), i + 1)
    plt.imshow(img.cpu())
    plt.title(f"t={t}")
    plt.axis("off")

plt.suptitle("Forward Diffusion Process (Clean → Noise)", fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## Train the CNN Denoiser

Now let's train our UNet-style CNN to learn the reverse diffusion process on CIFAR-10! 

**Key differences from MNIST:**
- **Input channels**: 3 (RGB) instead of 1 (grayscale)
- **Base channels**: 64 instead of 32 (more capacity for complex color images)
- **Image size**: 32×32 instead of 28×28

Everything else stays the same - same training pipeline, same loss function, same noise schedule! 🎯

In [ ]:
from src.diffusion_playground.models import CNNDenoiser
from src.diffusion_playground.training.denoiser_trainer import train_denoiser

# Data is already loaded as cifar_data tensor!
print(f"Training data shape: {cifar_data.shape}")

In [ ]:
# Create the CNN denoiser model for RGB images
model = CNNDenoiser(
    in_channels=3,  # RGB (3 channels) instead of grayscale (1 channel)
    base_channels=64,  # More capacity than MNIST (was 32)
    time_emb_dim=128  # Same time embedding dimension
)

# Print model summary
print(f"\nModel architecture:")
print(model)
print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# Train the model (using the same generic training function!)
# Note: CIFAR-10 is more complex than MNIST, may need longer training
# The training will automatically RESUME from the latest checkpoint if interrupted!
train_denoiser(
    model=model,
    data=cifar_data,
    noise_schedule=schedule,
    epochs=100,
    lr=1e-3,
    batch_size=128,
    checkpoint_dir="checkpoints/cifar10_cnn",
    save_every=5,
    resume=True
)

### 🔄 Auto-Resume Feature

The training will **automatically resume** from the latest checkpoint if interrupted! This is perfect for:
- ⏰ Google Colab sessions that time out
- 🔌 Unexpected disconnections
- 🛑 Manual interruptions

Just re-run the training cell and it will pick up where it left off. No manual checkpoint management needed!

## Test the Reverse Diffusion Process

Let's test our trained model by generating RGB images from pure noise! This will verify that all interfaces work correctly for CIFAR-10 before we move to long training on Google Colab.

In [ ]:
from src.diffusion_playground.training.denoiser_trainer import load_checkpoint

# Load checkpoint for testing
cp_name = "best_model.pt"
checkpoint_path = f"checkpoints/cifar10_cnn/{cp_name}"

# Create a fresh model instance
model = CNNDenoiser(in_channels=3, base_channels=64, time_emb_dim=128)
model.to(device)

# Load the checkpoint
checkpoint_info = load_checkpoint(model, checkpoint_path, device=device)
print(f"Loaded model trained for {checkpoint_info['epoch']} epochs")
print(f"Training loss: {checkpoint_info['loss']:.6f}")

In [ ]:
# Setup for generation
model.eval()
num_samples = 16
time_steps = schedule.time_steps

# Start from pure noise (RGB)
xt = torch.randn(num_samples, 3, 32, 32).to(device)

print(f"Starting reverse diffusion from noise shape: {xt.shape}")

# Reverse diffusion loop with STOCHASTIC sampling (adds noise for diversity!)
with torch.no_grad():
    for t in reversed(range(1, time_steps + 1)):
        # Create time tensor for all samples
        t_tensor = torch.full((num_samples,), t, device=device, dtype=torch.long)

        # Predict the noise
        pred_noise = model(xt, t_tensor)

        # Get schedule parameters
        beta_t = schedule.betas[t - 1]
        alpha_t = schedule.alphas[t - 1]
        alpha_bar_t = schedule.alpha_bars[t - 1]

        # Compute the mean of the reverse distribution
        mean = (xt - beta_t / torch.sqrt(1 - alpha_bar_t) * pred_noise) / torch.sqrt(alpha_t)

        # Add noise (STOCHASTIC sampling for diversity!)
        if t > 1:
            # Add Gaussian noise scaled by beta
            noise = torch.randn_like(xt)
            sigma_t = torch.sqrt(beta_t)
            x_prev = mean + sigma_t * noise
        else:
            # No noise at the final step
            x_prev = mean

        # Update xt
        xt = x_prev

        # Print progress every 100 steps
        if t % 100 == 0:
            print(f"  Step {time_steps - t + 1}/{time_steps} (t={t})")

print("Reverse diffusion complete!")

# Visualize generated RGB samples
fig, axes = plt.subplots(4, 4, figsize=(8, 8))
for idx, ax in enumerate(axes.flat):
    # Convert from [-1, 1] to [0, 1] for visualization
    img = (xt[idx].permute(1, 2, 0) + 1) / 2
    img = torch.clamp(img, 0, 1)

    ax.imshow(img.cpu())
    ax.axis("off")

# Create informative title with checkpoint details
title = f"Generated CIFAR-10 Images - {cp_name}\n"
title += f"Epoch: {checkpoint_info['epoch']} | Loss: {checkpoint_info['loss']:.6f}"
plt.suptitle(title, fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()